## Tiempos

Hago una limpieza de los datos que obtuve sobre los tiempos de ejecución de cada contrato, con las distintas herramientas y configuraciones.

Como resultado obtenemos dos dataframes distintas por cada benchmark (una para los tiempos en modo Epa y otra para los tiempos en modo States) y las mismas se exportan en formato .csv para poder visualizar y analizar los resultados. De esta manera se pueden comparar los tiempos de cada herramienta.

En otro notebook se hace un análisis de la cantidad de estados y transiciones de estados encontradas, viendo los grafos resultantes de ejecutar cada herramienta con sus distintas configuraciones posibles.

##### 0. Setup

In [1]:
import json5 as json
import pandas as pd

##### 1. Importo los resultados (formato JSON) y los guardo en dataframes de pandas.

In [2]:
with open('../results/benchmarks/echidna_benchmark1_times') as file:
    echidna_times_benchmark_1 = json.load(file)
    df_echidna_1 = pd.DataFrame.from_dict(echidna_times_benchmark_1)

with open('../results/benchmarks/verisol_benchmark1_times') as file:
    verisol_times_benchmark_1 = json.load(file)
    df_verisol_1 = pd.DataFrame.from_dict(verisol_times_benchmark_1)
    df_verisol_1.insert(4, 'reduce combinations', True)

with open('../results/benchmarks/echidna_benchmark2_times') as file:
    echidna_times_benchmark_2 = json.load(file)
    df_echidna_2 = pd.DataFrame.from_dict(echidna_times_benchmark_2)

with open('../results/benchmarks/verisol_benchmark2_times') as file:
    verisol_times_benchmark_2 = json.load(file)
    df_verisol_2 = pd.DataFrame.from_dict(verisol_times_benchmark_2)
    df_verisol_2.insert(4, 'reduce combinations', True)

##### 2. Defino algunas funciones que voy a usar para manipular las dataframes y hacer que los resultados sean más fáciles de leer.

In [3]:
df_echidna_2

,contract,test_limit,mode,reduce combinations,time_in_s
0,Auction,1000,e,False,14.71
1,Crowdfunding,1000,e,False,8.61
2,Crowdfunding,1000,s,False,9.86
3,EPXCrowdsale,1000,e,False,91.62
4,EPXCrowdsale,1000,s,False,13.61
...,...,...,...,...,...
109,EPXCrowdsale,50000,e,True,96.87
110,EPXCrowdsale,500000,e,True,403.60
111,Crowdfunding,1000,e,True,10.94
112,Crowdfunding,50000,e,True,31.60


In [4]:
def agregar_columnas(df):
    """ Le agregamos las columnas donde se van a guardar los tiempos de cada ejecución para cada contrato. """
    for mode in ['Epa', 'States']:
        for tx_bound in [4, 8]:
            df[f'Ver {mode} Time {tx_bound}'] = None

    for mode in ['Epa', 'States']:
        for test_limit in [1000, 50000, 500000]:
            df[f'Ech {mode} Time {test_limit}'] = None

In [5]:
def ordenar_resultados_echidna(df_joined, df_nueva):
    for index, row in df_joined.iterrows():
        contract = row['contract']
        test_limit = row['test_limit']
        mode = row['mode_e']
        time_in_s = row['time_in_s_e']
        
        
        if test_limit == 1000:
            if mode == 'e':
                df_nueva.at[index, 'Ech Epa Time 1000'] = time_in_s
            elif mode == 's':
                df_nueva.at[index, 'Ech States Time 1000'] = time_in_s
        elif test_limit == 50000:
            if mode == 'e':
                df_nueva.at[index, 'Ech Epa Time 50000'] = time_in_s
            elif mode == 's':
                df_nueva.at[index, 'Ech States Time 50000'] = time_in_s
        elif test_limit == 500000:
            if mode == 'e':
                df_nueva.at[index, 'Ech Epa Time 500000'] = time_in_s
            elif mode == 's':
                df_nueva.at[index, 'Ech States Time 500000'] = time_in_s
        elif test_limit == 1000000:
            if mode == 'e':
                df_nueva.at[index, 'Ech Epa Time 1000000'] = time_in_s
            elif mode == 's':
                df_nueva.at[index, 'Ech States Time 1000000'] = time_in_s
        else:
            print(f"Contract: {contract}")
            print(f"Test Limit: {test_limit}")
            print(f"Mode: {mode}")
            print(f"Time in s: {time_in_s}")
            df_nueva.at[index, 'EXTRA'] = time_in_s
            # raise Exception(f'El test_limit ({test_limit}) no matcheó con ninguno de los valores esperados...')

    df_nueva.drop(columns=['mode_e', 'test_limit', 'time_in_s_e'], inplace=True)



In [6]:
def ordenar_resultados_verisol(df_joined, df_nueva):
    for index, row in df_joined.iterrows():
        contract = row['contract']
        tx_bound = row['txBound']
        mode = row['mode_v']
        time_in_s = row['time_in_s_v']
        
        if tx_bound == 4:
            if mode == 'e':
                df_nueva.at[index, 'Ver Epa Time 4'] = time_in_s
            elif mode == 's':
                df_nueva.at[index, 'Ver States Time 4'] = time_in_s
        elif tx_bound == 8:
            if mode == 'e':
                df_nueva.at[index, 'Ver Epa Time 8'] = time_in_s
            elif mode == 's':
                df_nueva.at[index, 'Ver States Time 8'] = time_in_s
        else:
            print(f"Contract: {contract}")
            print(f"Tx Bound: {tx_bound}")
            print(f"Mode: {mode}")
            print(f"Time in s: {time_in_s}")
            df_nueva.at[index, 'EXTRA'] = time_in_s
            raise Exception(f'El txBound ({tx_bound}) no matcheó con ninguno de los valores esperados...')
    
    df_nueva.drop(columns=['mode_v', 'txBound', 'time_in_s_v', 'timeout', 'reduce combinations_v'], inplace=True)

##### 3. Aplico las funciones sobre los dataframes.

In [7]:
def process(df_echidna, df_verisol):

    # Hacemos un outer join para unir los dos dataframes por el campo 'contract.
    df_joined_temp = pd.merge(df_verisol, df_echidna, on='contract', how='outer', suffixes=('_v', '_e'))

    # Le agregamos las columnas donde se van a guardar los tiempos de cada ejecución para cada contrato.
    agregar_columnas(df_joined_temp)

    # Buscamos los resultados de entre todas las filas y los metemos en la columna que le corresponde.
    df_joined = df_joined_temp
    ordenar_resultados_echidna(df_joined_temp, df_joined)
    ordenar_resultados_verisol(df_joined_temp, df_joined)

    df_joined_grouped_by_contract = df_joined.groupby(['contract', 'reduce combinations_e'])
    df_joined_grouped_by_contract = df_joined_grouped_by_contract.first()
    # df_joined_grouped_by_contract.head()
    return df_joined_grouped_by_contract

In [8]:
df_joined_grouped_by_contract_1 = process(df_echidna_1, df_verisol_1)
df_joined_grouped_by_contract_2 = process(df_echidna_2, df_verisol_2)

In [9]:
df_joined_grouped_by_contract_2

Ver Epa Time 4   
contract                        reduce combinations_e                  
Auction                         False                           3.12  \
                                True                            3.12   
AuctionEnded                    False                           40.3   
                                True                            40.3   
AuctionWithdraw                 False                           67.9   
                                True                            67.9   
Crowdfunding                    False                          56.67   
                                True                           56.67   
CrowdfundingTime_Base           False                          39.45   
                                True                           39.45   
CrowdfundingTime_BaseBalance    False                         142.92   
                                True                          142.92   
CrowdfundingTime_BaseBalanceFix False                         133.18   
                                True                          133.18   
EPXCrowdsale                    False                         111.15   
                                True                          111.15   
EPXCrowdsaleIsCrowdsaleClosed   False                         260.88   
                                True                          260.88   
EscrowVault                     False                           87.0   
                                True                            87.0   
RefundEscrow                    False                          67.18   
                                True                           67.18   
RefundEscrowWithdraw            False                         162.39   
                                True                          162.39   
RockPaperScissors               False                           33.0   
                                True                            33.0   
SimpleAuction                   False                           2.34   
                                True                            2.34   
SimpleAuctionEnded              False                          128.9   
                                True                           128.9   
SimpleAuctionHB                 False                           None   
SimpleAuctionTime               False                         180.16   
                                True                          180.16   

                                                      Ver Epa Time 8   
contract                        reduce combinations_e                  
Auction                         False                           2.06  \
                                True                            2.06   
AuctionEnded                    False                          40.25   
                                True                           40.25   
AuctionWithdraw                 False                          69.77   
                                True                           69.77   
Crowdfunding                    False                          45.94   
                                True                           45.94   
CrowdfundingTime_Base           False                          37.07   
                                True                           37.07   
CrowdfundingTime_BaseBalance    False                        1350.96   
                                True                         1350.96   
CrowdfundingTime_BaseBalanceFix False                         132.31   
                                True                          132.31   
EPXCrowdsale                    False                         305.58   
                                True                          305.58   
EPXCrowdsaleIsCrowdsaleClosed   False                         814.31   
                                True                          814.31   
EscrowVault                     False                          98.76   
                     

In [10]:
df_epa_1 = df_joined_grouped_by_contract_1.filter(regex='Epa')

# De states tal vez tenemos 2 resultados de ejecuciones iguales (porque el reduceCombinations nunca se ejecuta).
# Por eso calculamos el mean. 
df_states_1 = df_joined_grouped_by_contract_1.filter(regex='States')
df_states_1 = df_states_1.groupby(['contract']).mean()


# Round the float values of both dataframes to 2 decimals
df_epa_1 = df_epa_1.applymap(lambda x: round(x, 2) if isinstance(x, float) else x)
df_states_1 = df_states_1.applymap(lambda x: round(x, 2) if isinstance(x, float) else x)


In [11]:
df_epa_2 = df_joined_grouped_by_contract_2.filter(regex='Epa')

# De states tenemos 2 resultados de ejecuciones iguales (porque el reduceCombinations nunca se ejecuta).
# Por eso calculamos el mean. 
df_states_2 = df_joined_grouped_by_contract_2.filter(regex='States')
df_states_2 = df_states_2.groupby(['contract']).mean()


# Round the float values of both dataframes to 2 decimals
df_epa_2 = df_epa_2.applymap(lambda x: round(x, 2) if isinstance(x, float) else x)
df_states_2 = df_states_2.applymap(lambda x: round(x, 2) if isinstance(x, float) else x)

In [12]:
# Remove column Ech Epa Time 1000000 from df_epa_2 and from df_states_2
# First check if the column exists
if 'Ech Epa Time 1000000' in df_epa_2.columns:
    df_epa_2.drop(columns=['Ech Epa Time 1000000'], inplace=True)

if 'Ech States Time 1000000' in df_states_2.columns:
    df_states_2.drop(columns=['Ech States Time 1000000'], inplace=True)

KeyError: "['Ech Epa Time 1000000'] not found in axis"

In [13]:
df_epa_2

Ver Epa Time 4   
contract                        reduce combinations_e                   
Auction                         False                            3.12  \
                                True                             3.12   
AuctionEnded                    False                           40.30   
                                True                            40.30   
AuctionWithdraw                 False                           67.90   
                                True                            67.90   
Crowdfunding                    False                           56.67   
                                True                            56.67   
CrowdfundingTime_Base           False                           39.45   
                                True                            39.45   
CrowdfundingTime_BaseBalance    False                          142.92   
                                True                           142.92   
CrowdfundingTime_BaseBalanceFix False                          133.18   
                                True                           133.18   
EPXCrowdsale                    False                          111.15   
                                True                           111.15   
EPXCrowdsaleIsCrowdsaleClosed   False                          260.88   
                                True                           260.88   
EscrowVault                     False                           87.00   
                                True                            87.00   
RefundEscrow                    False                           67.18   
                                True                            67.18   
RefundEscrowWithdraw            False                          162.39   
                                True                           162.39   
RockPaperScissors               False                           33.00   
                                True                            33.00   
SimpleAuction                   False                            2.34   
                                True                             2.34   
SimpleAuctionEnded              False                          128.90   
                                True                           128.90   
SimpleAuctionHB                 False                             NaN   
SimpleAuctionTime               False                          180.16   
                                True                           180.16   

                                                       Ver Epa Time 8   
contract                        reduce combinations_e                   
Auction                         False                            2.06  \
                                True                             2.06   
AuctionEnded                    False                           40.25   
                                True                            40.25   
AuctionWithdraw                 False                           69.77   
                                True                            69.77   
Crowdfunding                    False                           45.94   
                                True                            45.94   
CrowdfundingTime_Base           False                           37.07   
                                True                            37.07   
CrowdfundingTime_BaseBalance    False                         1350.96   
                                True                          1350.96   
CrowdfundingTime_BaseBalanceFix False                          132.31   
                                True                           132.31   
EPXCrowdsale                    False                          305.58   
                                True                           305.58   
EPXCrowdsaleIsCrowdsaleClosed   False                          814.31   
                                True                           814.31   
EscrowVault                     False  

##### 4. Exporto los resultados.

In [15]:
containing_dir = '../results/tables'

df_epa_1.to_csv(f'{containing_dir}/benchmark1_times_epa.csv')
df_states_1.to_csv(f'{containing_dir}/benchmark1_times_states.csv')

df_epa_2.to_csv(f'{containing_dir}/benchmark2_times_epa.csv')
df_states_2.to_csv(f'{containing_dir}/benchmark2_times_states.csv')